In [5]:
import os
# from dotenv import find_dotenv, load_dotenv
import openai
from langchain_openai import OpenAI
from langchain.agents import Tool, load_tools, create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate

In [6]:
from config import set_environment
set_environment()

In [7]:
# load_dotenv(find_dotenv())
# openai.api_key = os.getenv("OPENAI_API_KEY")

#==== Using OpenAI Chat API =======
llm_model = "gpt-3.5-turbo"
llm = OpenAI(temperature=0.0)

In [36]:
# test simple math operation
# query = "what is 3.1^2.1? show your calculations"
# query = "what is 3 + 1?"
query = "what is the capital of Canada?"

result = llm.invoke(query)
print(result)



The capital of Canada is Ottawa.


In [21]:
tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [28]:
print(tools[0].name, tools[0].description)

Calculator Useful for when you need to answer questions about math.


In [21]:
tools[0]

Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=OpenAI(client=<openai.resources.completions.Completions ob

In [30]:
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

In [31]:
prompt = PromptTemplate.from_template(template)

In [32]:
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=3 # to avoid high bills from the LLM
)

In [33]:
query = "If James is currently 45 years old, how old will he be in 50 years? \
    If he has 4 kids and adopted 7 more, how many children does he have?"

result = agent_executor.invoke({"input": query})

print(result['output'])



> Entering new AgentExecutor chain...
 I need to use the calculator to solve this problem.
Action: Calculator
Action Input: 45 + 50Answer: 9595 is the age James will be in 50 years.
Action: Calculator
Action Input: 4 + 7Answer: 1111 is the total number of children James has.
Final Answer: James will be 95 years old in 50 years and he has 11 children.

> Finished chain.
James will be 95 years old in 50 years and he has 11 children.


In [34]:
query = "what is 3.1^2.1? show your calculations"

result = agent_executor.invoke({"input": query})

print(result['output'])



> Entering new AgentExecutor chain...
 I should use a calculator to solve this problem.
Action: Calculator
Action Input: 3.1^2.1Answer: 10.761171606099687I now know the final answer.
Final Answer: 10.761171606099687

> Finished chain.
10.761171606099687


In [35]:
query = "what is the capital of Canada?"

result = agent_executor.invoke({"input": query})

print(result['output'])



> Entering new AgentExecutor chain...
 I should use a search engine to find the answer
Action: Search engine
Action Input: "capital of Canada"Search engine is not a valid tool, try one of [Calculator].I should use a calculator to find the answer
Action: Calculator
Action Input: None

ValueError: LLMMathChain._evaluate("
None
") raised error: unsupported expression type: <class 'NoneType'>. Please try again with a valid numerical expression